<a href="https://colab.research.google.com/github/htcysl/PCA-on-RGB-nDSM/blob/main/ReducedDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation necessary library


Rasterio: access to geospatial raster data

In [1]:
!pip install -q rasterio

In [2]:
%cd /content/drive/MyDrive/4.sınıfBahar/MipMap/DATASET/Alula/
%ls

/content/drive/MyDrive/4.sınıfBahar/MipMap/DATASET/Alula
10cm/      30cm/             DSM/              Ortho/                  Tile_10cm/  Tile_5cm/
15cm/      5cm/              DTM/              path_to_save_image.jpg  Tile_15cm/
20cm/      CombinedData/     nDSM/             ReducedData/            Tile_20cm/
20cm.txt/  datasetInfo.gdoc  nDSM_normalized/  TextOutput/             Tile_30cm/


In [3]:
import rasterio
import numpy as np
import os
import torch
from sklearn.decomposition import PCA
from rasterio.transform import from_origin
from sklearn.preprocessing import StandardScaler

Functions for reading and writing raster data.

In [4]:
def read_raster(file_path):
  with rasterio.open(file_path) as src:
    data = src.read()
    profile = src.profile
    return data, profile

def write_raster(file_path, data, profile):
  with rasterio.open(file_path, 'w', **profile) as dst:
    dst.write(data, 1)


The following for loop takes DSM and DTM files and then calculates the nDSM.

In [ ]:
file_list = ['A1','A2','B1','B2','B3','B4','C1','C2','C3','C4','D1','D2','D3','D4']

nDSM = DSM - DTM
save nDSM driver

In [ ]:
maximum = 10
minimum = -52.16815

for file in file_list:
  dsm_path = f'DSM/DSM_{file}.tif'
  dtm_path = f'DTM/DTM_{file}.tif'

  dsm, dsm_profile = read_raster(dsm_path)
  dtm, dtm_profile = read_raster(dtm_path)

  if dsm.shape != dtm.shape:
    raise ValueError("DSM and DTM rasters must have the same dimensions")

  ndsm = dsm - dtm

  ndsm[ndsm > 10] = 10

  ndsm_profile = dsm_profile.copy()
  ndsm_profile.update(dtype=rasterio.float32)

  # ndsm_normalized = (ndsm - minimum) / (maximum - minimum)

  #print(ndsm_normalized.min(), ndsm_normalized.max())

  #print(ndsm_normalized)

  ndsm_path = f'nDSM/nDSM_{file}.tif'
  with rasterio.open(ndsm_path, 'w', **ndsm_profile) as dst:
    dst.write(ndsm.astype(rasterio.float32))

  print(f"nDSM normalized saved successfully to {ndsm_path}")


nDSM normalized saved successfully to nDSM/nDSM_A1.tif
nDSM normalized saved successfully to nDSM/nDSM_A2.tif
nDSM normalized saved successfully to nDSM/nDSM_B1.tif
nDSM normalized saved successfully to nDSM/nDSM_B2.tif
nDSM normalized saved successfully to nDSM/nDSM_B3.tif
nDSM normalized saved successfully to nDSM/nDSM_B4.tif
nDSM normalized saved successfully to nDSM/nDSM_C1.tif
nDSM normalized saved successfully to nDSM/nDSM_C2.tif
nDSM normalized saved successfully to nDSM/nDSM_C3.tif
nDSM normalized saved successfully to nDSM/nDSM_C4.tif
nDSM normalized saved successfully to nDSM/nDSM_D1.tif
nDSM normalized saved successfully to nDSM/nDSM_D2.tif
nDSM normalized saved successfully to nDSM/nDSM_D3.tif
nDSM normalized saved successfully to nDSM/nDSM_D4.tif


In [ ]:
dsm, dtm, ndsm, ndsm_normalized = None, None, None, None

In [ ]:
for file in file_list:

  ndsm_normalized_path = f'nDSM_normalized/nDSM_normalized_{file}.tif'
  ndsm_normalized, ndsm_normalized_profile = read_raster(ndsm_normalized_path)

  # ----------------------------------------------------------------------------
  orthofoto_path = f'Ortho/Ortho_{file}.tif'
  with rasterio.open(orthofoto_path) as ortho_dataset:
    ortho = ortho_dataset.read()  # read all bands (RGB)
    ortho_profile = ortho_dataset.profile
  # ----------------------------------------------------------------------------
  print("shape of ndsm_normalized:",ndsm_normalized.shape)
  print("shape of ortho:",ortho.shape)

  if ndsm_normalized.shape[1:] != ortho.shape[1:]:
    raise ValueError("Dimension of nDSM and othophoto do not match.")

  # ----------------------------------------------------------------------------
  combined_data = np.vstack((ortho, ndsm_normalized))
  print("shape of combined_data:",combined_data.shape)
  # ----------------------------------------------------------------------------

  # Get the dimensions
  height, width = combined_data.shape[1], combined_data.shape[2]
  half_height, half_width = height//2, width//2
  # ----------------------------------------------------------------------------

  # Define the slices for the four quadrants
  quadrants = {
      'top_left' : combined_data[:, :half_height, :half_width],
      'top_right' : combined_data[:, :half_height, half_width:],
      'bottom_left' : combined_data[:, half_height:,:half_width],
      'bottom_right' : combined_data[:, half_height:, half_width:]
  }
  # ----------------------------------------------------------------------------

  # Define the profile for the output file
  combined_profile = ortho_profile.copy()
  combined_profile.update({
      'count' : combined_data.shape[0], # Number of bands
      'height' : half_height,
      'width' : half_width,
      'dtype' : combined_data.dtype,
      'bigtiff' : 'Yes'
  })
  # ----------------------------------------------------------------------------

  #Save each quadrant
  i = 1
  for quadrant_name, quadrant_data in quadrants.items():
    combined_profile.update({
        'height' : quadrant_data.shape[1],
        'width'  : quadrant_data.shape[2]
    })
    quadrant_path = f"CombinedData/{file}_{i}.tif"
    with rasterio.open(quadrant_path,'w', **combined_profile) as dst:
      dst.write(quadrant_data)
    print(f"{quadrant_name} quadrant saved sucessfully to {quadrant_path}")
    i += 1

  ortho, ndsm_normalized, combined_data  = None, None, None
  # ----------------------------------------------------------------------------

shape of ndsm_normalized: (1, 33000, 25000)
shape of ortho: (3, 33000, 25000)
shape of combined_data: (4, 33000, 25000)
top_left quadrant saved sucessfully to CombinedData/A1_1.tif
top_right quadrant saved sucessfully to CombinedData/A1_2.tif
bottom_left quadrant saved sucessfully to CombinedData/A1_3.tif
bottom_right quadrant saved sucessfully to CombinedData/A1_4.tif
shape of ndsm_normalized: (1, 33000, 25000)
shape of ortho: (3, 33000, 25000)
shape of combined_data: (4, 33000, 25000)
top_left quadrant saved sucessfully to CombinedData/A2_1.tif
top_right quadrant saved sucessfully to CombinedData/A2_2.tif
bottom_left quadrant saved sucessfully to CombinedData/A2_3.tif
bottom_right quadrant saved sucessfully to CombinedData/A2_4.tif
shape of ndsm_normalized: (1, 33000, 25000)
shape of ortho: (3, 33000, 25000)
shape of combined_data: (4, 33000, 25000)
top_left quadrant saved sucessfully to CombinedData/B1_1.tif
top_right quadrant saved sucessfully to CombinedData/B1_2.tif
bottom_left q

In [5]:
combined_list =  os.listdir('CombinedData/')

In [ ]:
print(new_profile)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 12500, 'height': 16500, 'count': 3, 'crs': CRS.from_epsg(32637), 'transform': Affine(0.05, 0.0, 392400.0,
       0.0, -0.05, 2943300.0), 'blockysize': 32, 'tiled': False, 'compress': 'lzw', 'interleave': 'pixel'}


In [8]:
import numpy as np
from osgeo import gdal
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import tifffile as tiff

# Read the 4-channel TIFF file
for file in combined_list:
    dataset = gdal.Open('CombinedData/'+file)

    # Check if the file was loaded correctly
    if dataset is None:
        raise FileNotFoundError(f"Could not open file {file}")

    # Extract the channels
    channels = [dataset.GetRasterBand(i+1).ReadAsArray() for i in range(dataset.RasterCount)]
    data = np.stack(channels, axis=-1)
    dataset = None
    channels = None

    # Reshape data to 2D array (pixels x channels)
    height, width, num_channels = data.shape
    data_reshaped = data.reshape(-1, num_channels)
    data = None

    # Standardize the data
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data_reshaped)
    data_reshaped = None
    scaler = None

    # Apply PCA to reduce the number of channels
    pca = PCA(n_components=3)
    data_pca = pca.fit_transform(data_scaled)
    data_scaled = None
    pca = None

    # Reshape back to the original dimensions (height x width x reduced channels)
    data_pca_reshaped = data_pca.reshape(height, width, -1)
    data_pca = None

    # Save the transformed data to a new 3-channel TIFF file
    output_tif = f'ReducedData/ReducedData_{file}'


    tiff.imwrite(output_tif, data_pca_reshaped.astype(np.float32))

    print(f'Saved 3-channel TIFF file to {output_tif}')
    data_pca_reshaped = None


Saved 3-channel TIFF file to ReducedData/ReducedData_A1_1.tif
Saved 3-channel TIFF file to ReducedData/ReducedData_A1_2.tif
Saved 3-channel TIFF file to ReducedData/ReducedData_A1_3.tif
Saved 3-channel TIFF file to ReducedData/ReducedData_A1_4.tif
Saved 3-channel TIFF file to ReducedData/ReducedData_A2_1.tif
Saved 3-channel TIFF file to ReducedData/ReducedData_A2_2.tif
Saved 3-channel TIFF file to ReducedData/ReducedData_A2_3.tif
Saved 3-channel TIFF file to ReducedData/ReducedData_A2_4.tif
Saved 3-channel TIFF file to ReducedData/ReducedData_B1_1.tif
Saved 3-channel TIFF file to ReducedData/ReducedData_B1_2.tif
Saved 3-channel TIFF file to ReducedData/ReducedData_B1_3.tif
Saved 3-channel TIFF file to ReducedData/ReducedData_B1_4.tif
Saved 3-channel TIFF file to ReducedData/ReducedData_B2_1.tif
Saved 3-channel TIFF file to ReducedData/ReducedData_B2_2.tif
Saved 3-channel TIFF file to ReducedData/ReducedData_B2_3.tif
Saved 3-channel TIFF file to ReducedData/ReducedData_B2_4.tif
Saved 3-

In [ ]:
with rasterio.open(output_tif, 'w', driver='GTiff', height=height, width=width, count=3, dtype=gdal.GDT_Float32) as dst:
    for i in range(3):
        dst.GetRasterBand(i+1).WriteArray(data_pca_reshaped[:,:,i])
        dst.GetRasterBand(i+1).SetNoDataValue(0)
        dst.GetRasterBand(i+1).SetMetadata({'count': 3, 'dtype': 'float32'})
        dst.GetRasterBand(i+1).SetMetadata({'height': height, 'width': width})
        dst.GetRasterBand(i+1).SetMetadata({'transform': dst.GetGeoTransform()})
        dst.GetRasterBand(i+1).SetMetadata({'crs': dst.GetProjection()})
        dst.GetRasterBand(i+1).SetMetadata({'driver': 'GTiff'})
        dst.GetRasterBand(i+1).SetMetadata({'compress': 'lzw'})
        dst.GetRasterBand(i+1).SetMetadata({'interleave': 'pixel'})
        dst.GetRasterBand(i+1).SetMetadata({'tiled': False})
        dst.GetRasterBand(i+1).SetMetadata({'blockxsize': 256, 'blockysize': 256})
        dst.GetRasterBand(i+1).SetMetadata({'interleave': 'pixel'})
        dst.GetRasterBand(i+1).SetMetadata({'interleave_fmt': 'byte'})
        dst.GetRasterBand(i+1).SetMetadata({'interleave_type': 1})


In [ ]:
import rasterio

# Function to read and print raster file metadata
def print_raster_metadata(file_path):
    with rasterio.open(file_path) as src:
        print("Raster Metadata:")
        print(f"Driver: {src.driver}")
        print(f"Width: {src.width}")
        print(f"Height: {src.height}")
        print(f"Count: {src.count}")
        print(f"Data Type: {src.dtypes[0]}")
        print(f"CRS: {src.crs}")
        print(f"Transform: {src.transform}")

        # Read the data and compute statistics
        data = src.read(1)
        print(f"Min Value: {data.min()}")
        print(f"Max Value: {data.max()}")
        print(f"Mean Value: {data.mean()}")
        print(f"Standard Deviation: {data.std()}")

# File path to the nDSM
ndsm_file = 'nDSM/nDSM_A1_2.tif'

# Print metadata of the nDSM file
print_raster_metadata(ndsm_file)


Raster Metadata:
Driver: GTiff
Width: 25000
Height: 33000
Count: 1
Data Type: float32
CRS: EPSG:32637
Transform: | 0.05, 0.00, 391150.00|
| 0.00,-0.05, 2946600.00|
| 0.00, 0.00, 1.00|
Min Value: -52.16815185546875
Max Value: 34.24493408203125
Mean Value: 0.6434074640274048
Standard Deviation: 2.5044519901275635
